<a href="https://colab.research.google.com/github/reggiebain/data-projects/blob/main/Physics_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Physics Chat-Bot
- In this notebook, we create a basic chat bot to respond to basic queries about important details/deadlines in a class that uses Canvas as an LMS.
- The primary functions are: basic Wolfram Alpha queries, asking for test dates/homework dates from Canvas
- Note: You'll need your own Canvas API key and course information as well as an API key for WolframAlpha and/or OpenAI

Train to recognize:
1. Math question for Wolfram
2. Class question for canvas
3. Wording question for chatgpt

In [ ]:
!pip install wolframalpha
!pip install openai
!pip install canvasapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 8.7 MB/s eta 0:00:00
  Created wheel for canvasapi: filename=canvasapi-3.0.0-py3-none-any.whl size=109644 sha256=1817378d

In [ ]:
import openai
import re
import wolframalpha
import requests
import json
from canvasapi import Canvas
from datetime import date, timedelta

# Set up Canvas API
CANVAS_API_URL = "<MY CANVAS API URL>" 
CANVAS_API_KEY = "<MY CANVAS API KEY>"

headers = {"Authorization": "Bearer " + CANVAS_API_KEY}

# Set up the OpenAI API key
openai.api_key = "<MY OPENAI KEY>

# Set up Wolfram Alpha API access
app_id = "<MY WOLFRAM API KEY>"
client = wolframalpha.Client(app_id)

In [ ]:
from datetime import datetime, date, timedelta
from dateutil import parser
#test.due_at.split('T')[0]

# Function that returns assignments due in certain date range.
def due_in_date_range(course, prompt):
  date_format = '%Y-%m-%dT%H:%M:%S%z'
  assignments = course.get_assignments()
#  from datetime import timedelta, datetime, date
  today = date.today()
  #.strftime("%B %d, %Y") # get todays date in nice format
  
  # Extract dates or "this week" or "next week"
  date_words = ['due']
  recent_assignments = []
  for word in date_words:
    if word in date_words:
      for assignment in assignments:
        if assignment.due_at:
          due_date = datetime.strptime(str(assignment.due_at), date_format).date()
          if today - timedelta(weeks=1) <= due_date <= today + timedelta(weeks=1):
            if assignment.published: # Show only published assignments
              recent_assignments.append(f'Assignment: {assignment.name}  |  Due Date: {due_date}')
  return_string = 'The assignments with +/- 1 Week from Today are as follows: \n' + '\n'.join(recent_assignments)
  return return_string   

In [ ]:
def get_exam_dates(course):
  
  # Get all assignments from the specified course
  assignments = course.get_assignments()
  
  # Specify date format and prepare to return in readable format
  date_format = '%Y-%m-%dT%H:%M:%S%z'
  
  tests = {}
  for assignment in assignments:
    if re.search(r'Exam.*\*\*', assignment.name):
      
      # Get and format the due date of the exams in a readable way
      due_date = datetime.strptime(str(assignment.due_at), date_format)
      readable_date = str(due_date.strftime("%A, %B %d, %Y"))
      
      # Get and format the exam name in a readable way
      readable_name = re.sub(r'\*\*$', '', assignment.name)
      
      # Add entry to the dictionary of exams to be printed
      tests[f'{readable_name}'] = readable_date

  return_string = (
       f"There are 4 exams and 1 final exam this semester whose\n" 
       f"dates are subject to change. As of now, the tests are \n"
       f"scheduled as follows:\n"
       f"{('Exam'):<14} {('Date'):<14}\n"
       f"--------------------------\n"
       f"{('Exam 1'):<14} {tests['Exam 1']:<14}\n"
       f"{('Exam 2'):<14} {tests['Exam 2']:<14}\n"
       f"{('Exam 3'):<14} {tests['Exam 3']:<14}\n"
       f"{('Exam 4'):<14} {tests['Exam 4']:<14}\n"
       f"{('Final Exam'):<14} {tests['Final Exam']:<14}\n"
       )
  return return_string

canvas = Canvas(CANVAS_API_URL, CANVAS_API_KEY)
residential = canvas.get_course(4638)
print(get_exam_dates(residential))

'There are 4 exams and 1 final exam this semester whose
dates are subject to change. As of now, the tests are 
scheduled as follows
Exam           Date          
--------------------------
Exam 1         Tuesday, February 21, 2023
Exam 2         Thursday, March 09, 2023
Exam 3         Wednesday, March 29, 2023
Exam 4         Wednesday, April 26, 2023
Final Exam     Tuesday, May 23, 2023



In [ ]:
from datetime import datetime, date, timedelta
from dateutil import parser
# Define a function to send a message to the ChatGPT API and get a response
def get_gpt_response(prompt):
    # Send the prompt to the API
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Extract the text from the response
    message = response.choices[0].text
    message = re.sub('[^0-9a-zA-Z\n\.\?,!]+', ' ', message).strip()

    return message

# Get a response from canvas
def get_canvas_response(question):
  canvas = Canvas(CANVAS_API_URL, CANVAS_API_KEY)
  user = canvas.get_user(4878) # Grab my own user id (reggiebain)
  courses = user.get_courses(enrollment_status='active')
  residential = canvas.get_course(4638)
  stem_foundations = canvas.get_course(4271)

  return due_in_date_range(residential, question)

# Answer question using Wolfram Alpha
def get_wolfram_response(question):
    try:
        res = client.query(question)
        answer = next(res.results).text
    except:
        answer = "Sorry, I don't know the answer to that question."
    return answer

# Define a function to handle user input and output chatbot responses
def run_chatbot():
    print("Hello! I'm a Chatbot. How can I help you?")
    while True:
        user_input = input("--> ")
        if user_input.lower() in ["quit", 'exit', 'bye']:
            break
        
        ask_chat = True
        canvas_keywords = ['canvas', 'test', 'hw', 'homework', 'our class', 'physics', 'due']
        for word in canvas_keywords:
          if word in user_input.lower():
            ask_chat = False
            prompt = str(user_input)
            response = get_canvas_response(prompt)
        
        wolfram_keywords = ['wolfram', 'alpha', 'solve', 'prime', 'solution', '*', '^2', 'units']
        for word in wolfram_keywords:
          if word in user_input.lower():
            ask_chat = False
            prompt = str(user_input)
            response = get_wolfram_response(prompt)
        
        if ask_chat:
          prompt = f'User: {user_input}\n Chatbot:'
          response = get_gpt_response(prompt)
        #prompt = f"User: {user_input}\nChatbot:"
        #response = get_gpt_response(prompt)
        #prompt = str(user_input)
        #response = get_wolfram_response(prompt)
        print(response)

# Run the chatbot
run_chatbot()


Hello! I'm a Chatbot. How can I help you?
--> quit


In [ ]:
import wolframalpha

while True:
  client = wolframalpha.Client('<MY WOLFRAM KEY>')
  prompt = str(input('How can I help you?'))
  if prompt == 'quit':
    break
  answer = client.query(prompt)
  output = next(answer.results).text
  print(output)

In [ ]:
print(datetime.date.today() + datetime.timedelta(weeks=2))

2023-04-06


Now we'll create some training data. Goals will be basic:
1. Questions about syllabus
2. Questions about due dates
3. Questions about grades

In [ ]:
# Create training data of tags, responses, patterns in json format for now for ease of typing
{'intents': [
    {'tag': 'grade',
     'patterns': ['what is my grade', 'i want to know my class grade','what is my course grade', 'what grade do I have', 'course grade'],
     'responses': ['You current course grade is posted on Canvas under an assignment named "Overall Course Grade". I am not authorized to give out grades here.'],
    },
    {'tag': 'policy',
     'patterns': ['How much do exams count for', 'How much is homework worth', 'how much are exams worth', 'how much of my grade is homework', 
                  'what percent of the grade is homework', 'how much of my grade is this exam', 'what is the final exam worth', 
                  'how much does the final exam count','what percent of my grade is the'],
     'responses': [(f"{('Item'):<14} {('Grade Pct.'):<14}\n"
       f"--------------------------\n"
       f"{('Exams (x4)'):<14} {('11.25%'):<14}\n"
       f"{('Final Exam'):<14} {('15%'):<14}\n"
       f"{('Lab'):<14} {('20%'):<14}\n"
       f"{('WebAssign'):<14} {('20%'):<14}\n\n"
       f"To calculate your grade, use a weighted average of your overall scores\n" 
       f"for each exam, lab, and WebAssign homework using the weights shown above\n"
       f"and your grades, which are all accessible on Canvas/WebAssign. I recommend\n"
       f"entering in projected scores for future exams if you have not yet taken those\n"
       f"rather than trying to calculate a partial grade. Enter exam scores as numbers\n"
       f"between 0 and 100. For example:\n\n"
       f"Grade = 0.45*(Exam1 + Exam2 + Exam3 + Exam4) + 0.15*(Final) + 0.2*(Homework) + 0.2*(Lab)"
       )]
    }
]}

In [ ]:
syllabus_responses = {}
syllabus_responses['grades'] = (f"{('Item'):<14} {('Grade Pct.'):<14}\n"
       f"--------------------------\n"
       f"{('Exams (x4)'):<14} {('11.25%'):<14}\n"
       f"{('Final Exam'):<14} {('15%'):<14}\n"
       f"{('Lab'):<14} {('20%'):<14}\n"
       f"{('WebAssign'):<14} {('20%'):<14}\n\n"
       f"To calculate your grade, use a weighted average of your overall scores\n" 
       f"for each exam, lab, and WebAssign homework using the weights shown above\n"
       f"and your grades, which are all accessible on Canvas/WebAssign. I recommend\n"
       f"entering in projected scores for future exams if you have not yet taken those\n"
       f"rather than trying to calculate a partial grade. Enter exam scores as numbers\n"
       f"between 0 and 100. You can ask me to do this calculation! For example:\n\n"
       f"Grade = 0.45*(Exam1 + Exam2 + Exam3 + Exam4) + 0.15*(Final) + 0.2*(Homework) + 0.2*(Lab)"
       )
print(syllabus_responses['grades'])

Item           Grade Pct.    
--------------------------
Exams (x4)     11.25%        
Final Exam     15%           
Lab            20%           
WebAssign      20%           

To calculate your grade, use a weighted average of your overall scores
for each exam, lab, and WebAssign homework using the weights shown above
and your grades, which are all accessible on Canvas/WebAssign. I recommend
entering in projected scores for future exams if you have not yet taken those
rather than trying to calculate a partial grade. Enter exam scores as numbers
between 0 and 100. For example:

Grade = 0.45*(Exam1 + Exam2 + Exam3 + Exam4) + 0.15*(Final) + 0.2*(Homework) + 0.2*(Lab)


In [ ]:
canvas_responses = {}

canvas_responses['tests'] = (f"There are 4 exams and 1 final exam this semester whose\n" 
       f"dates are subject to change. As of now, the tests are scheduled as follows\n"
       f"{('Exam'):<14} {('Date'):<14}\n"
       f"--------------------------\n"
       f"{('Exam 1'):<14} {tests['Exam 1']:<14}\n"
       f"{('Final Exam'):<14} {('15%'):<14}\n"
       f"{('Lab'):<14} {('20%'):<14}\n"
       f"{('WebAssign'):<14} {('20%'):<14}\n\n"
       f"To calculate your grade, use a weighted average of your overall scores\n" 
       f"for each exam, lab, and WebAssign homework using the weights shown above\n"
       f"and your grades, which are all accessible on Canvas/WebAssign. I recommend\n"
       f"entering in projected scores for future exams if you have not yet taken those\n"
       f"rather than trying to calculate a partial grade. Enter exam scores as numbers\n"
       f"between 0 and 100. For example:\n\n"
       f"Grade = 0.45*(Exam1 + Exam2 + Exam3 + Exam4) + 0.15*(Final) + 0.2*(Homework) + 0.2*(Lab)"
       )